In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import binom
from typing import Callable

In [ ]:
def call_payoff(spot: np.ndarray, strike: np.ndarray) -> np.ndarray:
    return np.maximum(spot - strike, 0.0)

def put_payoff(spot: np.ndarray, strike: np.ndarray) -> np.ndarray:
    return np.maximum(strike - spot, 0.0)

In [ ]:
# European at first
def binomial(
    spot: float,
    strike: float,
    rate: float,
    volatility: float,
    dividend: float,
    expiry: float,
    steps: int,
    payoff: Callable) -> float:

    # For `steps` periods on the asset price tree:
    # there will be `nodes = steps + 1 ` terminal price nodes
    nodes = steps + 1

    # For an asset price tree with `steps` periods the length of a single period is `h`
    h = expiry / steps

    # Calculate the up factor `u` and the down factor `d` from the given input data
    u = np.exp((rate - dividend) * h + np.sqrt(h) * volatility)
    d = np.exp((rate - dividend) * h - np.sqrt(h) * volatility)

    # Pre-calculate the discount factor to get the present value of the option
    disc = np.exp(-rate * expiry)

    # The risk-neutral probability of an up move `pu` and a down move `pd` (its complement)
    pu = (np.exp((rate - dividend) * h) - d) / (u - d)
    pd = 1.0 - pu

    # Start a counter variable for the option price initialized at zero
    prc_t = 0.0

    # Loop over the terminal nodes starting at the top and move downwards
    for i in range(nodes):

        # The spot price at node `i`
        spot_t = spot * (u ** (steps - i) * (d ** i))

        # The risk-neutral probability of node `i`
        prob_t = binom.pmf(steps - i, steps, pu)

        # The option payoff at node `i` weighted by the risk-neutral
        # probability of node `i`
        # NB: we are keeping a running sum (when the loop ends it will
        # equal the summation of the risk-neutral probability weighted option
        # payoff values)
        prc_t += payoff(spot_t, strike) * prob_t

    # Calculate the present value of the option and return to the caller
    prc_t *= disc

    return prc_t

In [ ]:
# This is a canonical problem in McDonald
# See Chapter 10, p. 309 (Figure 10.5)
spot = 41.0
strike = 40.0
rate = 0.08
vol = 0.30
expiry = 1.0
div = 0.0
steps = 3

In [ ]:
# Call the function and confirm that the price is $\approx$ $7.074
binomial(spot, strike, rate, vol, div, expiry, steps, call_payoff)

In [ ]:
# Loop over successively larger number of timesteps in the tree
# looking for convergence to the Black-Scholes price ($9.96)
for i in range(10, 300, 10):
    prc = binomial(spot, strike, rate, vol, div, expiry, i, call_payoff)
    print(f"({i}, {prc})")